In [126]:
from py2neo import authenticate, Graph, Node, Relationship
import getpass
import db_vis1
import re
import numpy as np

In [181]:
options = {"Disease":'name','Patient':'name','Term':'name','PatientMaritalStatus':'name',
           'Gender':'name','District':'name','Age':'name'}
db_vis1.draw(topDs,options,"/home/putus/working_directory/hackathon/resultZ_")

In [2]:
uname = "neo4j"
passw = "beta"

In [3]:
authenticate("localhost:7474",uname,passw)
graph = Graph("http://localhost:7474/db/data/hackthon") # create instances of a Graph() class 

# How much Data we have in DataBASE

In [188]:
Q1 = '''
match (n) return head(labels(n)) as label, count(*)
'''
data1 = graph.run(Q1)

In [189]:
for d1 in data1:
    print(d1)

('label': 'Patient', 'count(*)': 98)
('label': 'PatientMaritalStatus', 'count(*)': 6)
('label': 'Disease', 'count(*)': 268)
('label': 'Age', 'count(*)': 51)
('label': 'District', 'count(*)': 4)
('label': 'Gender', 'count(*)': 2)
('label': 'Term', 'count(*)': 163)


# top State

In [199]:
q = '''
match (u:District) with u,size( ()-[]->(u)) 
as topDisease order by topDisease desc limit 5 return u.name as label, topDisease
'''
da = graph.run(q)

In [200]:
for i in da:
    print(i)

('label': 'Jaipur', 'topDisease': 62)
('label': 'Jodhpur', 'topDisease': 18)
('label': 'Udaipur', 'topDisease': 12)
('label': 'Jaisalmer', 'topDisease': 6)


# The Top 10 Disease

In [160]:
Q2 = '''
match (u:Disease) with u,size( ()-[]->(u)) 
as topDisease order by topDisease desc limit 5 return u.name as label, topDisease
'''
data2 = graph.run(Q2)

In [193]:
topDisease

['disorder ',
 'diabetes ',
 'diabetes mellitus ',
 'rheumatoid arthritis ',
 'arthritis ']

In [161]:
topDisease = []
for d1 in data2:
    print(d1)
    topDisease.append(re.findall(r"'(.*?)'",str(d1), re.DOTALL)[1])

('label': 'disorder ', 'topDisease': 50)
('label': 'diabetes ', 'topDisease': 28)
('label': 'diabetes mellitus ', 'topDisease': 27)
('label': 'rheumatoid arthritis ', 'topDisease': 21)
('label': 'arthritis ', 'topDisease': 21)


# common Disease in district Udaipur

In [49]:
def query2graph(query):
    list_result=[]
    resultQuery = graph.run(query)
    for data in resultQuery:
        list_result.append(data)
    return list_result

In [43]:
def findCommonPatient(cmDisease):
    ResultQuery = graph.run(cmDisease)
    common_patient = []
    for data in ResultQuery:
        val = str(data)
        val = re.findall('"([^"]*)"',val)
        common_patient.append(val[0])
    return common_patient

In [128]:
def greatethanAge(cmDisease):
    ResultQuery = graph.run(cmDisease)
    common_patient = []
    for data in ResultQuery:
        val = str(data)
        val = re.findall('"([^"]*)"',val)
        common_patient.append([val[0],val[1]])
    return (np.array(common_patient))

# common Disease in district Udaipur

In [182]:
Q = '''
match (u:Patient)-[r1]->(t) where t.name = 'Udaipur' RETURN u AS source_node, 
id(u) AS source_id, r1, t AS target_node, id(t) AS target_id
'''

In [183]:
list_result = []
list_result = query2graph(Q)
for pat in findCommonPatient(Q):
    Qx = "match (u:Patient)-[r1]->(t:Disease) where u.name = '"+pat+"' RETURN u AS source_node,id(u) AS source_id, r1, t AS target_node, id(t) AS target_id"
    res = graph.run(Qx)
    for dt in res:
            list_result.append(dt)

In [184]:
db_vis1.draw(list_result,options,"/home/putus/working_directory/hackathon/resultX_")

# Common Disease age greater than 60

In [69]:
# Patient greater than age 60

In [185]:
QY = '''
match (u:Patient)-[r1]->(t:Age) where t.name > "60.0" RETURN u AS source_node, 
id(u) AS source_id, r1, t AS target_node, id(t) AS target_id
'''

In [186]:
list_result1 = []
list_result1 = query2graph(QY)
for pat in findCommonPatient(QY):
        Qy = "match (u:Patient)-[r1]->(t:Disease) where u.name = '"+pat+"' RETURN u AS source_node,id(u) AS source_id, r1, t AS target_node, id(t) AS target_id"
        res = graph.run(Qy)
        c = 2
        for dt in res:
            if c>0:
                list_result1.append(dt)
                c=c-1
        Qz = "match (u:Patient)-[r1]->(t:District) where u.name = '"+pat+"' RETURN u AS source_node,id(u) AS source_id, r1, t AS target_node, id(t) AS target_id"
        res = graph.run(Qz)
        for dis in res:
            list_result1.append(dis)

In [187]:
db_vis1.draw(list_result1,options,"/home/putus/working_directory/hackathon/resultY_")

# top Disease and District, age, sex

In [177]:
f=[]
topDs = []
for item in topDisease:
    QD = "match (u:Patient)-[r1]->(t:Disease) where t.name = '"+item+"' RETURN u AS source_node,id(u) AS source_id, r1, t AS target_node, id(t) AS target_id"
    res = graph.run(QD)
    for dt in res:
        topDs.append(dt)
    for pat in findCommonPatient(QD):
        f.append(pat)

In [178]:
patt = sorted(set(f), key=lambda x:f.index(x))

In [179]:
for pa in patt:
    QQ1 = "match (u:Patient)-[r1]->(t:District) where u.name = '"+pa+"' RETURN u AS source_node,id(u) AS source_id, r1, t AS target_node, id(t) AS target_id"
    res = graph.run(QQ1)
    for tt in res:
        topDs.append(tt)
    QQ2 = "match (u:Patient)-[r1]->(t:Age) where u.name = '"+pa+"' RETURN u AS source_node,id(u) AS source_id, r1, t AS target_node, id(t) AS target_id"
    res = graph.run(QQ2)
    for tt in res:
        topDs.append(tt)
    QQ3 = "match (u:Patient)-[r1]->(t:Gender) where u.name = '"+pa+"' RETURN u AS source_node,id(u) AS source_id, r1, t AS target_node, id(t) AS target_id"
    res = graph.run(QQ3)
    for tt in res:
        topDs.append(tt)